# Run stock-price prediction model

## Notebook ([`papermill`](https://papermill.readthedocs.io/en/latest/index.html)) parameters

In [1]:
run_time = '2019-10-01T10:00'
stop_minutes = 10
dir = None
earliest_data_cutoff = '2019-04-01'
ticker = 'AAPL'
batch_size = 100000
max_epochs = 10000

## Date/Time helpers

In [2]:
from sys import executable as python
from datetime import datetime as dt, date, time, timedelta as Δ
from dateutil.parser import parse

strptime = dt.strptime
now = dt.now()
today = now.date()

if run_time:
    run_time = parse(run_time)
else:
    run_time = dt.now()

run_date = run_time.date()

if stop_minutes:
    stop_at = run_time + Δ(minutes=stop_minutes)
else:
    stop_at = None

earliest_data_cutoff = parse(earliest_data_cutoff)
earliest_data_cutoff_date = earliest_data_cutoff.date()
    
print('Running at %s (stopping within %dmins)' % (run_time.strftime('%Y-%m-%d %H:%M:%S'), stop_minutes))

Running at 2019-10-01 10:00:00 (stopping within 10mins)


## Initialize `data`, `models` directories

In [3]:
from pathlib import Path
if not dir:
    dir = Path.cwd()

data_dir = dir / 'data'
data_dir.mkdir(parents=True, exist_ok=True)
models_dir = dir / 'models'

## Read IEX API credentials
Search for a secret key in:
- `$IEX_SECRET_KEY` env var
- `iex.secret_key` in a config file located at:
  - `$CONFIG_PATH` if set
  - `~/.config/iex.ini` otherwise

In [4]:
from os import environ as env

api = 'https://cloud.iexapis.com'

SECRET_KEY_VAR = 'IEX_SECRET_KEY'
CONFIG_PATH_VAR = 'CONFIG_PATH'

if SECRET_KEY_VAR in env:
    secret_key = env[SECRET_KEY_VAR]
else:
    if CONFIG_PATH_VAR in env:
        config_path = Path(env[CONFIG_PATH_VAR])
    else:
        config_path = Path.home() / '.config' / 'iex.ini'

    if not config_path.exists():
        raise Exception("No %s env var, and config path %s doesn't exist" % (SECRET_KEY_VAR, config_path))

    from configparser import ConfigParser
    config = ConfigParser()
    config.read(str(config_path))
    iex_config = config['iex']
    secret_key = iex_config['secret_key']

## Initialize `dates` and `minutes` that trading occurs during

In [5]:
start_minute = time(9, 30)
end_minute = time(16, 0)
def get_minutes():
    minute = start_minute
    while minute < end_minute:
        yield minute
        hr = minute.hour
        min = minute.minute
        min += 1
        if min == 60:
            min = 0
            hr += 1
        minute = time(hr, min)

minutes = list(get_minutes())
num_minutes = len(minutes); num_minutes

def get_dates(start_date, end_date, step=1):
    date = start_date
    while date != end_date:
        # only emit weekdays
        if date.weekday() <= 4:
            yield date
        date += Δ(days=step)

start_date = earliest_data_cutoff_date
end_date = run_date + Δ(days=1)
dates = list(get_dates(start_date, end_date))
num_dates = len(dates)

print(
    'Fetching/Processing data from %d days ([%s,%s)), %d minutes per day' % (
        num_dates, 
        start_date.strftime('%Y-%m-%d'),
        end_date.strftime('%Y-%m-%d'),
        num_minutes,
    )
)

Fetching/Processing data from 132 days ([2019-04-01,2019-10-02)), 390 minutes per day


In [6]:
def get_ticker_dir(ticker):
    return data_dir / ticker

def get_ticker_date_path(ticker, date):
    return get_ticker_dir(ticker) / date.strftime('%Y%m%d')

In [7]:
!{python} -m pip install -Uq requests
from requests import get as GET

In [8]:
import json

def fetch(date, ticker, refetch_partial=False):
    date_str = date.strftime('%Y%m%d')
    out_path = get_ticker_date_path(ticker, date)
    refetch = False
    prev_data = None
    if out_path.exists():
        if refetch_partial:
            with out_path.open('r') as f:
                prev_data = json.load(f)
                if len(prev_data) < num_minutes:
                    refetch = True
                    print(
                        'Re-fetching data for %s from %s (found %d per-minute quotes instead of %d)' % (
                            ticker, 
                            date_str, 
                            len(prev_data), 
                            num_minutes
                        )
                    )
                else:
                    return True
        else:
            return True
    else:
        print('Fetching data for %s from %s' % (ticker, date_str))

    url = f'https://cloud.iexapis.com/stable/stock/{ticker}/chart/date/{date_str}?token={secret_key}'
    resp = GET(url)
    resp.raise_for_status()
    data = json.loads(resp.content)
    if prev_data is None or len(data) > len(prev_data):
        with out_path.open('wb') as f:
            f.write(resp.content)

        if refetch:
            print('Re-fetch found data for %s %s' % (date_str, ticker))
        return True
    elif len(data) < len(prev_data):
        raise Exception('Found %d data, less than previous amount %d' % (len(data), len(prev_data)))
    else:
        print('Re-fetched %s, found same %d data' % (out_path, len(data)))

    return False

In [9]:
!{python} -m pip install -Uq joblib
from joblib import Parallel, delayed

Fetch quotes between a start point in the past (IEX seems to serve 6-7mos of historic data) and today

In [10]:
%%time

N = 32  # fetch parallelism
refetch_empty = False

Parallel(n_jobs=N)(
    delayed(fetch)(date, ticker, refetch_partial=refetch_empty or date == today)
    for date in dates 
); None

CPU times: user 574 ms, sys: 204 ms, total: 778 ms
Wall time: 8.31 s


In [11]:
!{python} -m pip install -Uq pandas matplotlib numpy scipy
import numpy as np
from numpy import \
    array, \
    nan, isnan as na, \
    zeros, \
    count_nonzero as cnz, \
    mean, std, \
    unique, \
    logical_and as l_and, \
    logical_or as l_or, \
    exp, log, sqrt
from numpy.random import shuffle, permutation
from pandas import concat, DataFrame as DF, read_csv, read_json
import pandas as pd
from scipy.stats import describe

These are the columns that we receive from IEX, describing trades that took place there:

In [12]:
features = [ 'open', 'close', 'high', 'low', 'average', 'volume', 'notional', 'numberOfTrades' ]

In [13]:
def load_ticker_date_df(date, ticker, latest_data_datetime=None):
    date_str = date.strftime('%Y%m%d')
    out_path = get_ticker_date_path(ticker, date)
    if not out_path.exists():
        return None
    df = read_json(out_path)
    if df.empty:
        return None
    # Convert "date", "minute" columns into a single "datetime" column
    df['datetime'] = df['date'].apply(lambda d: d.strftime('%Y-%m-%d')) + ' ' + df['minute']
    df['datetime'] = df['datetime'].apply(lambda s: strptime(s, '%Y-%m-%d %H:%M'))
    df.drop(columns=['date', 'minute'])
    if latest_data_datetime:
        df = df.loc[df.datetime <= latest_data_datetime]
    
    # Drop other columns ("market"-data columns, which are updated on a 15-min delay)
    df = df[['datetime'] + features]

    df.set_index('datetime', inplace=True)
    df.sort_index(inplace=True)

    return df

In [14]:
def load_ticker_df(ticker, N=None, limit=None):
    if limit is None:
        ds = dates
    elif type(limit) == int:
        ds = dates[:limit]
    elif type(limit) == date:
        ds = [ date for date in dates if date < limit ]        
    elif type(limit) == dt:
        ds = [ date for date in dates if date < limit.date() ]
    else:
        raise Exception('Unrecognized limit: %s' % limit)

    if N is None:
        df = concat([ load_ticker_date_df(date, ticker, latest_data_datetime=run_time) for date in ds ])
    else:
        df = concat(Parallel(n_jobs=N)( delayed(load_ticker_date_df)(date, ticker) for date in ds ))
    
    for col in features:
        # "quantity" features (volume, numberOfTrades, notional) are sometimes -1 instead of NaN
        df[col] = df[col].apply(lambda n: nan if n < 0 else n)

    return df

In [15]:
%%time
aapl = load_ticker_df('AAPL'); aapl

CPU times: user 3.93 s, sys: 83.1 ms, total: 4.02 s
Wall time: 4.1 s


,open,close,high,low,average,volume,notional,numberOfTrades
datetime,,,,,,,,
2019-04-01 09:30:00,191.645,190.650,191.645,190.600,191.189,4320,825935.940,44
2019-04-01 09:31:00,190.700,190.980,190.980,190.640,190.761,3246,619210.510,32
2019-04-01 09:32:00,191.060,190.930,191.090,190.780,190.951,2253,430211.740,30
2019-04-01 09:33:00,190.980,190.830,191.010,190.760,190.946,2241,427911.290,27
2019-04-01 09:34:00,190.760,190.700,190.760,190.600,190.666,1069,203822.465,12
...,...,...,...,...,...,...,...,...
2019-10-01 09:56:00,228.130,227.805,228.130,227.780,227.929,4582,1044370.725,69
2019-10-01 09:57:00,227.835,227.875,227.970,227.710,227.795,4487,1022115.850,30
2019-10-01 09:58:00,227.915,227.945,227.950,227.845,227.907,9520,2169678.980,42


## Fill missing data
Minutes where no trades were recorded can be assigned prices based on previous minutes' closing prices

We don't attempt to cross day boundaries, as the moves overnight are often much larger than minute-to-minute moves during the day, and skew the data.

Make a series representing minutes that are the first of their day (i.e. 9:30am)

In [16]:
prev_datetime = aapl.index.to_series().shift(1)
datetime = aapl.index.to_series()

def to_date(dt):
    return dt.date()

day_start = datetime.apply(to_date) != prev_datetime.apply(to_date)

Populate each minute with the previous minute's closing price

Assume each day's initial opening is the same as an imagined closing price from a minute prior (when trading wasn't open).

In [17]:
prev_close = aapl.close.shift(1)
prev_close[day_start] = aapl.open
aapl['prev_close'] = prev_close

Track minutes where no trades occurred:

In [18]:
nan_idxs = aapl[features].isna().any(axis=1); nan_idxs
nans = aapl[nan_idxs]
nans

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close
datetime,,,,,,,,,
2019-04-02 13:29:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,193.000
2019-04-05 13:36:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,196.540
2019-04-05 14:29:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,196.795
2019-04-15 13:33:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,198.910
2019-04-16 13:39:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,199.725
...,...,...,...,...,...,...,...,...,...
2019-09-30 13:39:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,223.930
2019-09-30 13:43:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,224.040
2019-09-30 13:44:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,NaN


Spot-check: consecutive pairs of minutes where no trades occurred

In [19]:
nan_pairs = l_and(nan_idxs, nan_idxs.shift(1).fillna(False))
concat([ aapl[nan_pairs], aapl[nan_pairs.shift(-1).fillna(False)] ]).sort_index()

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close
datetime,,,,,,,,,
2019-07-24 13:00:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,208.12
2019-07-24 13:01:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,NaN
2019-07-24 13:24:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,207.90
2019-07-24 13:25:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,NaN
2019-08-23 14:16:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,202.66
...,...,...,...,...,...,...,...,...,...
2019-09-27 13:59:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,NaN
2019-09-27 14:11:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,218.22
2019-09-27 14:12:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,NaN


Group by date, and forward-fill "previous closing" price.

For minutes when no trades occurred, we will treat the last valid closing price as both the opening and closing price. However, we don't persist these across day boundaries (this results in apparent large jumps at opening that skew training).

In [20]:
grouped = aapl.groupby(to_date)
aapl.prev_close = grouped.prev_close.apply(lambda s: s.fillna(method='ffill'))
aapl.count()

open              46670
close             46670
high              46670
low               46670
average           46670
volume            47038
notional          47038
numberOfTrades    47038
prev_close        47038
dtype: int64

Fill `NaN` openings with the previous valid closing price, which is interpreted as the start and end price for a minute where no activity occurred:

In [21]:
fill_cols = [ 'open', 'close', 'high', 'low', 'average', ]
for col in fill_cols:
    aapl[col].fillna(aapl.prev_close, inplace=True)
aapl.count()

open              47038
close             47038
high              47038
low               47038
average           47038
volume            47038
notional          47038
numberOfTrades    47038
prev_close        47038
dtype: int64

We expect that there are no remaining `NaN`s in the dataset (unless they appear during the first minute of the day; at the time of writing that has not been observed)

Spot check that `NaN` entries have been filled in based on the previous minute's closing price:

In [22]:
aapl[l_or(nan_idxs, nan_idxs.shift(-1).fillna(False))]

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close
datetime,,,,,,,,,
2019-04-02 13:28:00,192.990,193.000,193.050,192.990,193.014,1100,212316.000,11,192.990
2019-04-02 13:29:00,193.000,193.000,193.000,193.000,193.000,0,0.000,0,193.000
2019-04-05 13:35:00,196.535,196.540,196.540,196.535,196.537,205,40290.200,3,196.495
2019-04-05 13:36:00,196.540,196.540,196.540,196.540,196.540,0,0.000,0,196.540
2019-04-05 14:28:00,196.820,196.795,196.820,196.795,196.811,731,143868.985,13,196.820
...,...,...,...,...,...,...,...,...,...
2019-09-30 13:44:00,224.040,224.040,224.040,224.040,224.040,0,0.000,0,224.040
2019-09-30 14:08:00,224.050,224.050,224.060,224.050,224.053,300,67216.000,3,224.010
2019-09-30 14:09:00,224.050,224.050,224.050,224.050,224.050,0,0.000,0,224.050


## Log-fold transforms

Market-data time-series are frequently approximated as being log-normally distributed; the fold-change from one interval to the next is considered to be normally distributed.

Here we copy the price data above, make `open` a log-fold change over the previous `close`, and express other intra-minute features as a log-fold change over their corresponding `open`ing price:

In [23]:
lg = aapl.copy()
for col in fill_cols[1:]:
    lg[col] = log(lg[col] / lg.open)
lg.open = log(lg.open / lg.prev_close)

lg.drop(columns='prev_close', inplace=True)
lg[fill_cols][l_or(nan_idxs, nan_idxs.shift(1).fillna(False))]

,open,close,high,low,average
datetime,,,,,
2019-04-02 13:29:00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-04-02 13:30:00,0.000414,0.000000,0.000362,-0.000052,0.000104
2019-04-05 13:36:00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-04-05 13:37:00,-0.000102,-0.000051,0.000000,-0.000051,-0.000010
2019-04-05 14:29:00,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...
2019-09-30 13:45:00,-0.000045,-0.000045,0.000245,-0.000045,0.000013
2019-09-30 14:09:00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-09-30 14:10:00,0.000491,-0.000245,0.000000,-0.000245,-0.000138


In [24]:
lg.count()

open              47038
close             47038
high              47038
low               47038
average           47038
volume            47038
notional          47038
numberOfTrades    47038
dtype: int64

Normalize and rename some features:
- `trades` ⟶ `sqrt(numberOfTrades)` (variance-stabilizing transform for a Poisson distribution)
- `volume` ⟶ `avgVol` (divide out then number of trades, and `log`-transform to attempt to normalize outliers

In [25]:
lg.drop(columns=[ 'volume', 'notional', 'numberOfTrades' ], inplace=True)
lg['trades'] = sqrt(aapl.numberOfTrades)  # variance-stabilizing transform for a Poisson distribution
lg['avgVol'] = log(1 + (aapl.volume / aapl.numberOfTrades).fillna(0))  # volume-per-trade  seems extremely high-tailed, so apply log1p
lg.rename(columns={'average': 'avg', 'high': 'hi', 'low': 'lo'}, inplace=True)
lg

,open,close,hi,lo,avg,trades,avgVol
datetime,,,,,,,
2019-04-01 09:30:00,0.000000,-0.005205,0.000000,-0.005468,-0.002382,6.633250,4.596955
2019-04-01 09:31:00,0.000262,0.001467,0.001467,-0.000315,0.000320,5.656854,4.629253
2019-04-01 09:32:00,0.000419,-0.000681,0.000157,-0.001467,-0.000571,5.477226,4.332048
2019-04-01 09:33:00,0.000262,-0.000786,0.000157,-0.001153,-0.000178,5.196152,4.430817
2019-04-01 09:34:00,-0.000367,-0.000315,0.000000,-0.000839,-0.000493,3.464102,4.500735
...,...,...,...,...,...,...,...
2019-10-01 09:56:00,-0.000219,-0.001426,0.000000,-0.001535,-0.000881,8.306624,4.210731
2019-10-01 09:57:00,0.000132,0.000176,0.000592,-0.000549,-0.000176,5.477226,5.014406
2019-10-01 09:58:00,0.000176,0.000132,0.000154,-0.000307,-0.000035,6.480741,5.427883


Add `next_close` to each minute, which is the log-fold change of the next-minute's closing price over the current minute's.

This will be treated as the "label" for a training sample anchored at each minute: given the current minute's activity (and that of several minutes prior), can we predict the next minute's closing price?

The next minute's opening- and intra-minute features (high, low, average) are not trained on / learned; IEX publishes each minute's data about halfway through to the next minute, so predicting the next minute's closing price is a good goal for the purposes of this example.

In [26]:
grouped = lg.groupby(to_date)

def get_next_close(df):
    # the next minute's closing price is a combination of:
    # - the next minute's "open" field (log-fold change over current-minute close)
    # - the next minute's "close" field (log-fold change over next-minute open)
    df['next_close'] = df.open.shift(-1) + df.close.shift(-1)
    return df
lg = grouped.apply(get_next_close)
lg

,open,close,hi,lo,avg,trades,avgVol,next_close
datetime,,,,,,,,
2019-04-01 09:30:00,0.000000,-0.005205,0.000000,-0.005468,-0.002382,6.633250,4.596955,0.001729
2019-04-01 09:31:00,0.000262,0.001467,0.001467,-0.000315,0.000320,5.656854,4.629253,-0.000262
2019-04-01 09:32:00,0.000419,-0.000681,0.000157,-0.001467,-0.000571,5.477226,4.332048,-0.000524
2019-04-01 09:33:00,0.000262,-0.000786,0.000157,-0.001153,-0.000178,5.196152,4.430817,-0.000681
2019-04-01 09:34:00,-0.000367,-0.000315,0.000000,-0.000839,-0.000493,3.464102,4.500735,0.000786
...,...,...,...,...,...,...,...,...
2019-10-01 09:56:00,-0.000219,-0.001426,0.000000,-0.001535,-0.000881,8.306624,4.210731,0.000307
2019-10-01 09:57:00,0.000132,0.000176,0.000592,-0.000549,-0.000176,5.477226,5.014406,0.000307
2019-10-01 09:58:00,0.000176,0.000132,0.000154,-0.000307,-0.000035,6.480741,5.427883,0.000548


`next_close` was populated within each day above; that means the final minute of each day is missing a label, so we drop them:

In [27]:
lg.dropna(how='any', inplace=True)
lg.count()

open          46916
close         46916
hi            46916
lo            46916
avg           46916
trades        46916
avgVol        46916
next_close    46916
dtype: int64

## Persistent assignment of labeled samples to {training, validation} sets
Assigning samples to validation vs training should be persistent; we will train iteratively over time on the full history up to that point, and samples marked for validation should always be held out from training.

We achieve this by seeding a PRNG for each sample based on that sample's minute ID (`int(strftime('%Y%m%d%H%M'))`), and pulling a single random float. The set of samples whose floats are less than $V$ should represent $100V$% of the data, and can be used collectively as a validation-split representing proportion $V$ of the total set of labeled samples.

In [28]:
from random import random, seed
validation_split = 0.2

# decide whether a given minute should be treated as a validation (otherwise: training) test case
# seed PRNG so that this info is stable over time, and we never train on samples marked for "validation"
def get_validation_rand(dt):
    seed(int(dt.strftime('%Y%m%d%H%M')))
    return random()

datetime = lg.index.to_series()  # update index since we dropped the last minute of each day
vrs = datetime.apply(get_validation_rand).rename('validation_rand')
vfs = (vrs < validation_split).rename('validation?')
tn, vn = vfs[~vfs].count(), vfs[vfs].count()
n = tn + vn
tf, vf = tn / n, vn / n
print('%d training samples (%.2f%%), %d validation (%.2f%%)' % (tn, 100 * tf, vn, 100 * vf))

37388 training samples (79.69%), 9528 validation (20.31%)


Separate the log-transformed data into training and validation sets.

Also compute {$\mu$, $\sigma$} of the training sets, which will be used to standardize the data.

In [29]:
lg_t = lg[~vfs]
means = lg_t.mean(axis=0).rename('μ')
stddevs = lg_t.std(axis=0).rename('σ')
concat([ means, stddevs ], axis=1)

,μ,σ
open,0.000008,0.000226
close,-0.000007,0.000595
hi,0.000290,0.000439
lo,-0.000303,0.000470
avg,-0.000001,0.000374
trades,3.885767,1.768095
avgVol,4.440881,0.601695
next_close,-0.000002,0.000627


Helpers for showing descriptive statistics of each column of a DataFrame:

In [30]:
def desc(df):
    stats = DF([ describe(df[col]) for col in df.columns ]).set_index(df.columns)
    return desc_df(stats)

def desc_df(stats):
    stats['min'] = stats.minmax.apply(lambda t: t[0])
    stats['max'] = stats.minmax.apply(lambda t: t[1])
    stats['σ'] = sqrt(stats.variance)
    stats.rename(columns={ 'mean': 'μ', 'nobs': 'n' }, inplace=True)
    stats.drop(columns=[ 'minmax', 'variance' ], inplace=True)
    cs = list(stats.columns)
    stats = stats[cs[:2] + [ 'σ' ] + cs[2:-1]]
    return stats

In [31]:
desc(lg)

,n,μ,σ,skewness,kurtosis,min,max
open,46916,7.683632e-06,0.000225,-0.602067,19.441104,-0.006159,0.002457
close,46916,-5.545212e-06,0.000592,-0.112491,46.232458,-0.018992,0.014990
hi,46916,2.911537e-04,0.000441,4.846663,59.889062,0.000000,0.014990
lo,46916,-3.025504e-04,0.000462,-7.033265,186.934325,-0.023854,0.000000
avg,46916,-5.734251e-07,0.000371,-1.091739,77.219710,-0.015078,0.007264
trades,46916,3.888766e+00,1.768649,1.170360,3.268725,0.000000,21.679483
avgVol,46916,4.442022e+00,0.598586,-3.865561,28.774971,0.000000,10.533455
next_close,46916,-4.973731e-08,0.000626,-0.330888,38.263176,-0.018822,0.015714


The log-transform has captures the idea that, intuitively, a price is as likely to double as half. 

However, it's worth noting that the high kurtosis values imply that the data is not perfectly *normally distributed* after the log-transform; price movements are empirically "heavy-tailed", as we see here.

## Standardize data
Subtract and divide out the mean and stddev of the training features.

Note that:
- the training {$\mu$,$\sigma$} will be used for validation samples as well
- we store $-\mu/\sigma$ (pre-standardization $0$) in `zeros`, for use padding sliding windows later

In [32]:
normed = lg.copy()
zeros = -means / stddevs; zeros

for col in normed.columns:
    normed[col] = (normed[col] - means[col]) / stddevs[col]

trn = normed[~vfs]
val = normed[ vfs]
normed

,open,close,hi,lo,avg,trades,avgVol,next_close
datetime,,,,,,,,
2019-04-01 09:30:00,-0.036956,-8.732159,-0.661109,-10.990947,-6.363650,1.553923,0.259390,2.761125
2019-04-01 09:31:00,1.123692,2.476574,2.680911,-0.024090,0.858887,1.001693,0.313069,-0.414942
2019-04-01 09:32:00,1.816728,-1.131399,-0.303475,-2.475610,-1.521365,0.900098,-0.180877,-0.832908
2019-04-01 09:33:00,1.121991,-1.307922,-0.303326,-1.807417,-0.471903,0.741129,-0.016727,-1.084243
2019-04-01 09:34:00,-1.660843,-0.516478,-0.661109,-1.140184,-1.313469,-0.238486,0.099476,1.256790
...,...,...,...,...,...,...,...,...
2019-10-01 09:55:00,-0.425401,2.221946,2.535278,0.645622,1.071202,2.291442,0.602447,-2.620745
2019-10-01 09:56:00,-1.006941,-2.382851,-0.661109,-2.622049,-2.352128,2.500351,-0.382503,0.492732
2019-10-01 09:57:00,0.545890,0.306850,0.688181,-0.522338,-0.465317,0.900098,0.953183,0.492581


In [33]:
desc(normed)

,n,μ,σ,skewness,kurtosis,min,max
open,46916,-0.002948,0.995030,-0.602067,19.441104,-27.298231,10.838607
close,46916,0.002643,0.994646,-0.112491,46.232458,-31.890234,25.191701
hi,46916,0.002089,1.005445,4.846663,59.889062,-0.661109,33.482632
lo,46916,0.001724,0.982459,-7.033265,186.934325,-50.122002,0.645622
avg,46916,0.002474,0.991461,-1.091739,77.219710,-40.299058,19.421211
trades,46916,0.001696,1.000313,1.170360,3.268725,-2.197714,10.063779
avgVol,46916,0.001896,0.994834,-3.865561,28.774971,-7.380623,10.125691
next_close,46916,0.002616,0.998941,-0.330888,38.263176,-30.017619,25.065950


We continue to observe the presence of heavy outliers (≈10s of standard deviations, as evidenced by the `min`/`max` values in many columns). 

Dropping outliers seems unwise (sudden large price movements are arguably most important to be aware of!), so we'll proceed without further corrections.

Separate the `next_close` ("label") column, and name it `y`:

In [34]:
y = normed.next_close
normed.drop(columns='next_close', inplace=True)
normed

,open,close,hi,lo,avg,trades,avgVol
datetime,,,,,,,
2019-04-01 09:30:00,-0.036956,-8.732159,-0.661109,-10.990947,-6.363650,1.553923,0.259390
2019-04-01 09:31:00,1.123692,2.476574,2.680911,-0.024090,0.858887,1.001693,0.313069
2019-04-01 09:32:00,1.816728,-1.131399,-0.303475,-2.475610,-1.521365,0.900098,-0.180877
2019-04-01 09:33:00,1.121991,-1.307922,-0.303326,-1.807417,-0.471903,0.741129,-0.016727
2019-04-01 09:34:00,-1.660843,-0.516478,-0.661109,-1.140184,-1.313469,-0.238486,0.099476
...,...,...,...,...,...,...,...
2019-10-01 09:55:00,-0.425401,2.221946,2.535278,0.645622,1.071202,2.291442,0.602447
2019-10-01 09:56:00,-1.006941,-2.382851,-0.661109,-2.622049,-2.352128,2.500351,-0.382503
2019-10-01 09:57:00,0.545890,0.306850,0.688181,-0.522338,-0.465317,0.900098,0.953183


## Rolling window, numpy `ndarray` conversion
Create a numpy array from a rolling 30-minute window of the standardized features:

In [35]:
window = 30

cols = normed.columns
x = np.moveaxis(
    array([ 
        concat(
            [ 
                normed[col].shift(i)
                for i in reversed(range(window)) 
            ], 
            axis=1
        ) \
        .fillna(zeros[col]) \
        .values
        for col in cols
    ]),
    0, 2
)
x.shape    

(46916, 30, 7)

For each minute, we have the last 30-minutes of data about the 7 features.

Partial windows are padded with "0"s (pre-standardization 0; actually $-\mu/\sigma$ in post-standardization space, taken from the `zeros` array above)

Store training/validation data, and assign the full `x` and `y` datasets as a concatenation of training and validation samples, respectively.

In [36]:
tx = x[~vfs]
vx = x[ vfs]
ty = y[~vfs].to_numpy()
vy = y[ vfs].to_numpy()
[ a.shape for a in [ tx,ty, vx,vy, x,y ] ]

[(37388, 30, 7), (37388,), (9528, 30, 7), (9528,), (46916, 30, 7), (46916,)]

In [37]:
factor = exp(y * stddevs.next_close + means.next_close).rename('factor')
next_close = (aapl.close * factor).rename('next_close')
concat([ aapl, next_close, factor ], axis=1)

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close,next_close,factor
datetime,,,,,,,,,,,
2019-04-01 09:30:00,191.645,190.650,191.645,190.600,191.189,4320,825935.940,44,191.645,190.980,1.001731
2019-04-01 09:31:00,190.700,190.980,190.980,190.640,190.761,3246,619210.510,32,190.650,190.930,0.999738
2019-04-01 09:32:00,191.060,190.930,191.090,190.780,190.951,2253,430211.740,30,190.980,190.830,0.999476
2019-04-01 09:33:00,190.980,190.830,191.010,190.760,190.946,2241,427911.290,27,190.930,190.700,0.999319
2019-04-01 09:34:00,190.760,190.700,190.760,190.600,190.666,1069,203822.465,12,190.830,190.850,1.000787
...,...,...,...,...,...,...,...,...,...,...,...
2019-10-01 09:56:00,228.130,227.805,228.130,227.780,227.929,4582,1044370.725,69,228.180,227.875,1.000307
2019-10-01 09:57:00,227.835,227.875,227.970,227.710,227.795,4487,1022115.850,30,227.805,227.945,1.000307
2019-10-01 09:58:00,227.915,227.945,227.950,227.845,227.907,9520,2169678.980,42,227.875,228.070,1.000548


## Build model

In [38]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, SimpleRNN, Dense
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [39]:
def create_model():
    model = Sequential()
    model.add(SimpleRNN(4, input_shape=(window, len(cols))))
    model.add(Dense(1))
    model.build()
    model.compile(loss='mae', optimizer='adam')
    return model

In [40]:
import re
regex = '^\d{4}-\d\d-\d\dT\d\d:\d\d$'
ckpt_dirs = sorted([
    dir
    for dir in models_dir.iterdir() 
    if re.match(regex, dir.name)
])
model = create_model()
if ckpt_dirs:
    ckpt_dir = ckpt_dirs[-1]
    print('Loading model from %s' % ckpt_dir)
    ckpt = tf.train.latest_checkpoint(str(ckpt_dir))
    print('Found model %s' % ckpt)
    model.load_weights(ckpt)
else:
    print('No pre-existing models found; using untrained model')

model.summary()

Loading model from /Users/ryan/c/pipelines/notebooks/models/2019-09-01T10:00
Found model /Users/ryan/c/pipelines/notebooks/models/2019-09-01T10:00/cp-0060.ckpt
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 4)                 48        
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 53
Trainable params: 53
Non-trainable params: 0
_________________________________________________________________


As a baseline for evaluating our model's training progress, compute the mean absolute error we'd expect from a network that simply learned to predict the mean of the training labels for every sample:

In [41]:
mean(abs(ty - mean(ty))), mean(abs(vy - mean(vy)))

(0.6483054888744145, 0.6447829256786222)

In [42]:
def diffs_df(model):
    def stats(x, y, name=None, predict=True):
        suffix = ' (%s)' % name if name else ''
        if predict:
            predx = model.predict(x)[:, 0]
        else:
            predx = x

        diffs = (y - predx)
        mae = abs(diffs)
        mse = mae**2

        return DF({
            ('y^' + suffix): predx,
            ('Δ' + suffix): diffs,
            ('|Δ|' + suffix): mae,
            ('Δ²' + suffix): mse,
        })

    train = stats(tx, ty, 't')
    val = stats(vx, vy, 'v')
    predx_t = train['y^ (t)']
    predx_v = val['y^ (v)']
    predx = concat([ predx_t, predx_v ])
    all = stats(predx, np.concatenate([ ty, vy ]), name='*', predict=False)

    return concat([
        desc(train),
        desc(val),
        desc(all)
    ])

In [43]:
diffs_df(model)

,n,μ,σ,skewness,kurtosis,min,max
y^ (t),37388,0.016676,0.040312,0.726155,2.935196,-5.196213e-01,0.373915
Δ (t),37388,-0.016676,0.999594,-0.394659,44.450411,-3.020476e+01,24.971106
|Δ| (t),37388,0.648041,0.761248,5.707303,113.843974,1.133467e-06,30.204759
Δ² (t),37388,0.999440,6.811847,89.053034,10569.424385,1.284748e-12,912.327467
y^ (v),9528,0.016209,0.040320,0.822869,1.996866,-1.545926e-01,0.307487
Δ (v),9528,-0.003328,0.995355,-0.333994,15.170901,-1.229388e+01,13.188913
|Δ| (v),9528,0.644077,0.758857,3.979701,32.443473,5.146873e-07,13.188913
Δ² (v),9528,0.990639,4.105699,20.946421,663.569459,2.649030e-13,173.947434
y^ (*),46916,0.016581,0.040313,0.745777,2.743692,-5.196213e-01,0.373915
Δ (*),46916,-0.013965,0.998739,-0.382505,38.583847,-3.020476e+01,24.971106


In [44]:
model_ckpt_path = models_dir / run_time.strftime('%Y-%m-%dT%H:%M') / 'cp-{epoch:04d}.ckpt'; model_ckpt_path

PosixPath('/Users/ryan/c/pipelines/notebooks/models/2019-10-01T10:00/cp-{epoch:04d}.ckpt')

In [45]:
%%time
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=batch_size,
          epochs=max_epochs,
          callbacks=[
              EarlyStopping(patience=100, restore_best_weights=True),
              ModelCheckpoint(str(model_ckpt_path), save_weights_only=True, save_best_only=True, period=20),
          ]
         )
diffs_df(model)

Train on 37388 samples, validate on 9528 samples
Epoch 1/10000
37388/37388 [==============================] - 2s 45us/sample - loss: 0.6480 - val_loss: 0.6441
Epoch 2/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6480 - val_loss: 0.6441
Epoch 3/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6480 - val_loss: 0.6441
Epoch 4/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6480 - val_loss: 0.6441
Epoch 5/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6480 - val_loss: 0.6441
Epoch 6/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6480 - val_loss: 0.6441
Epoch 7/10000
37388/37388 [==============================] - 0s 5us/sample - loss: 0.6480 - val_loss: 0.6442
Epoch 8/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6480 - val_loss: 0.6442
Epoch 9/10000
37388/37388 [==============================] - 0s 6us/sample - l

Epoch 74/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6480 - val_loss: 0.6441
Epoch 75/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6480 - val_loss: 0.6441
Epoch 76/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6480 - val_loss: 0.6440
Epoch 77/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6480 - val_loss: 0.6440
Epoch 78/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6480 - val_loss: 0.6440
Epoch 79/10000
37388/37388 [==============================] - 0s 5us/sample - loss: 0.6480 - val_loss: 0.6440
Epoch 80/10000
37388/37388 [==============================] - 0s 7us/sample - loss: 0.6480 - val_loss: 0.6440
Epoch 81/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6480 - val_loss: 0.6440
Epoch 82/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6480 - val_loss: 0.6440
Epoch 83/1

Epoch 149/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 150/10000
37388/37388 [==============================] - 0s 5us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 151/10000
37388/37388 [==============================] - 0s 5us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 152/10000
37388/37388 [==============================] - 0s 5us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 153/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 154/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 155/10000
37388/37388 [==============================] - 0s 5us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 156/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 157/10000
37388/37388 [==============================] - 0s 5us/sample - loss: 0.6479 - val_loss: 0.6440
E

Epoch 223/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 224/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 225/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 226/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 227/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 228/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 229/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 230/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 231/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
E

Epoch 297/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 298/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 299/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 300/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 301/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 302/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 303/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 304/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 305/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
E

Epoch 371/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 372/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 373/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 374/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 375/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 376/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 377/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 378/10000
37388/37388 [==============================] - 0s 5us/sample - loss: 0.6479 - val_loss: 0.6440
Epoch 379/10000
37388/37388 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6440
E

,n,μ,σ,skewness,kurtosis,min,max
y^ (t),37388,0.016818,0.033248,1.091340,2.412276,-3.426946e-01,0.258578
Δ (t),37388,-0.016818,0.999414,-0.401365,44.291802,-3.015459e+01,24.957550
|Δ| (t),37388,0.647895,0.761139,5.698603,113.385581,3.627357e-08,30.154591
Δ² (t),37388,0.999085,6.797816,88.862924,10532.616734,1.315772e-15,909.299353
y^ (v),9528,0.016326,0.033009,1.125662,1.929972,-1.008291e-01,0.204280
Δ (v),9528,-0.003444,0.995138,-0.343298,15.188123,-1.235668e+01,13.169139
|Δ| (v),9528,0.644026,0.758616,3.981834,32.503591,9.951230e-06,13.169139
Δ² (v),9528,0.990207,4.105997,20.986514,665.903137,9.902697e-11,173.426222
y^ (*),46916,0.016718,0.033200,1.098249,2.316496,-3.426946e-01,0.258578
Δ (*),46916,-0.014102,0.998551,-0.389735,38.461160,-3.015459e+01,24.957550


In [ ]:
hist = DF(model.history.history)
hist.iloc[-110:-90]

In [ ]:
hist.iloc[-200:]

In [ ]:
w = model.get_weights(); w, [ l.shape for l in w ]

In [ ]:
describe(ty)